In [2]:
import torch
print(torch.cuda.is_available())      
print(torch.cuda.device_count())  

True
2


/home/jupyter-dai7591/.conda/envs/yolo/lib/python3.10/site-packages/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


### Load dataset

Dataset from: https://xviewdataset.org/
Full list of albumations functions in ultralytics: https://docs.ultralytics.com/reference/data/augment/h=albumentation#ultralyticsdataaugmentBaseTransform

`convert_xView.py`
Custom script that does the train/val split and converts geojson to labels.

`sanity_check`
Custom script that ensures each image and label pair exists.

### Inspect dataset

In [2]:
from pathlib import Path

# 1) explicitly point at your xView folder
root = Path("/home/jupyter-dai7591/yolo/datasets/xView")

image_exts = {".png", ".tif"}
label_exts = {".txt"}

print("xView")
print("Image counts:")
for split in ("train", "val"):
    img_dir = root / "images" / split
    if not img_dir.exists():
        print(f"  {split}: path not found ({img_dir})")
    else:
        cnt = sum(1 for f in img_dir.rglob("*") if f.suffix.lower() in image_exts)
        print(f"  {split}: {cnt}")

print("\nLabel counts:")
for split in ("train", "val"):
    lbl_dir = root / "labels" / split
    if not lbl_dir.exists():
        print(f"  {split}: path not found ({lbl_dir})")
    else:
        cnt = sum(1 for f in lbl_dir.rglob("*") if f.suffix.lower() in label_exts)
        print(f"  {split}: {cnt}")

xView
Image counts:
  train: 846
  val: 281

Label counts:
  train: 846
  val: path not found (/home/jupyter-dai7591/yolo/datasets/xView/labels/val)


### Load model

In [3]:
from ultralytics import YOLO

# Load a pretrained model
model = YOLO("test_yolo/experiment_14/weights/best.pt") # n for nano, x for huge

### Train model
Includes list of augmentations to apply to each batch before training loop. Doesn't create more images, just augments the existing ones (and labels accordingly) to enrich dataset. 

In [ ]:
results = model.train(
    data="datasets/xView/xView.yaml",  # Path to custom dataset YAML
    epochs=100,                          # Total number of training epochs
    imgsz=640,                           # Input image size for training
    split=0.2,
    patience=15,                         # patience: Number of epochs with no improvement before early stopping
    batch=4,                            # batch: Batch size (use -1 or a fraction for auto GPU memory utilization)
    save=True,                           # save: Save training checkpoints and final model weights
    save_period=8,                      # save_period: Frequency (in epochs) for saving checkpoints; -1 disables
    cache=False,                         # cache: Enable caching of dataset images (True for RAM, 'disk' for disk caching)
    device='0',                       # device: Specify the computational device (e.g., 0, "cpu", or list of GPUs)
    workers=8,                           # workers: Number of worker threads for data loading
    project="test_yolo",                 # project: Name of the project directory for saving outputs
    name="xView fine tune",                 # name: Name of the training run
    exist_ok=False,                      # exist_ok: Allow overwriting an existing project/name directory if True
    pretrained=False,                     # pretrained: Start from a pretrained model or provide a path to one
    optimizer="auto",                    # optimizer: Choice of optimizer (e.g., 'SGD', 'Adam', or 'auto')
    seed=18,                             # seed: Random seed for reproducibility
    single_cls=False,                    # single_cls: Treat all classes as a single class (useful for binary tasks)
    classes=None,                        # classes: List of class IDs to train on; None uses all classes
    rect=False,                          # rect: Enable rectangular training to minimize padding in batches
    multi_scale=True,                    # multi_scale: Enable multi-scale training by varying the imgsz during training
    cos_lr=False,                        # cos_lr: Use a cosine learning rate scheduler
    close_mosaic=10,                     # close_mosaic: Disable mosaic augmentation in the last N epochs
    resume=False,                        # resume: Resume training from the last checkpoint if available
    amp=True,                            # amp: Enable Automatic Mixed Precision training
    #fraction=1.0,                       # fraction: Fraction of the dataset to use for training
    freeze=None,                         # freeze: Freeze the first N layers or specify list of layers to freeze
    lr0=0.01,                            # lr0: Initial learning rate
    lrf=0.001,                           # lrf: Final learning rate as a fraction of lr0 (lr0 * lrf)
    momentum=0.937,                      # momentum: Momentum factor for the optimizer
    weight_decay=0.0005,                 # weight_decay: L2 regularization coefficient to prevent overfitting
    warmup_epochs=3.0,                   # warmup_epochs: Number of epochs for learning rate warmup
    warmup_momentum=0.8,                 # warmup_momentum: Initial momentum value during the warmup phase
    warmup_bias_lr=0.1,                  # warmup_bias_lr: Learning rate for bias parameters during warmup
    box=7.5,                             # box: Weight of the box loss component (bounding box regression)
    cls=0.5,                             # cls: Weight of the classification loss component
    dfl=1.5,                             # dfl: Weight of the distribution focal loss for fine-grained classification
    nbs=64,                              # nbs: Nominal batch size for loss normalization
    dropout=0.0,                         # dropout: Dropout rate for regularization
    val=True,                            # val: Enable validation during training
    plots=True,                          # plots: Generate and save plots of training and validation metrics
    hsv_h=0.015,  # Adjusts the hue of the image by a fraction of the color wheel
    hsv_s=0.7,    # Alters the saturation for varied color intensity
    hsv_v=0.4,    # Modifies brightness to simulate different lighting conditions
    degrees=0.0,  # Rotates the image 0° (no rotation); adjust between 0 and 180 for random rotation
    translate=0.1,  # Translates the image by 10% of its size
    scale=0.5,    # Scales the image, simulating objects at different distances
    shear=0.0,    # Shears the image 0° (no shearing); modify between -180 and 180 for effect
    perspective=0.0,  # Applies no perspective transformation; use values up to 0.001 if needed
    flipud=0.0,   # Probability (0%) of flipping the image upside down
    fliplr=0.5,   # 50% chance to horizontally flip the image
    bgr=0.0,      # No conversion from RGB to BGR (0% chance)
    mosaic=1.0,   # Enables mosaic augmentation by merging four images into one
    mixup=0.0,    # Disables mixup augmentation (blends two images)
    cutmix=0.0,   # Disables cutmix augmentation (combines portions of two images)
    erasing=0.4,   # Randomly erases regions (40% probability) to encourage robust feature extraction
)

New https://pypi.org/project/ultralytics/8.3.123 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.122 🚀 Python-3.10.17 torch-2.7.0+cu126 CUDA:0 (NVIDIA RTX 6000 Ada Generation, 48625MiB)


engine/trainer: task=obb, mode=train, model=test_yolo/experiment_14/weights/best.pt, data=datasets/xView/xView.yaml, epochs=100, time=None, patience=15, batch=8, imgsz=640, save=True, save_period=8, cache=False, device=0, workers=8, project=test_yolo, name=xView fine tune40, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=18, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=0.2, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimi

train: Scanning /home/jupyter-dai7591/yolo/datasets/xView/labels/train.cache... 846 images, 0 backgrounds, 0 corrupt: 100%|██████████| 846/846 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3402.6±1931.5 MB/s, size: 10958.9 KB)


val: Scanning /home/jupyter-dai7591/yolo/datasets/xView/labels/train.cache... 846 images, 0 backgrounds, 0 corrupt: 100%|██████████| 846/846 [00:00<?, ?it/s]


...

WARNING ⚠️ NMS time limit 2.400s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  65%|██████▌   | 69/106 [01:04<00:40,  1.08s/it]

WARNING ⚠️ NMS time limit 2.400s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  66%|██████▌   | 70/106 [01:09<00:35,  1.00it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.10 GiB. GPU 0 has a total capacity of 47.49 GiB of which 1.35 GiB is free. Process 997447 has 2.58 GiB memory in use. Process 1999272 has 5.10 GiB memory in use. Process 2670656 has 3.98 GiB memory in use. Process 2946540 has 758.00 MiB memory in use. Process 3198744 has 9.41 GiB memory in use. Process 898415 has 4.91 GiB memory in use. Including non-PyTorch memory, this process has 19.37 GiB memory in use. Of the allocated memory 15.77 GiB is allocated by PyTorch, and 3.04 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### Validated Model

In [12]:
# Validate the model
from ultralytics import YOLO

model = YOLO("../yolox/xView_2xGPU27/weights/epoch0.pt") # n for nano, x for huge
metrics = model.val(data="../datasets/xView/xview_yolo.yaml")  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category



Ultralytics 8.3.127 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA RTX 6000 Ada Generation, 48625MiB)
YOLO11x-obb summary (fused): 199 layers, 58,804,903 parameters, 0 gradients, 203.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 385.0±516.4 MB/s, size: 55.9 KB)


val: Scanning /home/jupyter-dai7591/yolo/datasets/xView/labels/val.cache... 1300 images, 1117 backgrounds, 0 corrupt: 100%|██████████| 2417/2417 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 152/152 [00:36<00:00,  4.21it/s]


                   all       2417      69508      0.396      0.281      0.223      0.138
   Fixed-wing Aircraft         12         12      0.478       0.25      0.155      0.106
        Small Aircraft         13         31      0.781      0.581       0.69       0.48
 Passenger/Cargo Plane         58        110      0.722      0.736      0.732      0.543
            Helicopter          2          4          0          0     0.0732     0.0586
     Passenger Vehicle         64        350          1          0    0.00548    0.00354
             Small Car        827      23836      0.655       0.77      0.734      0.449
                   Bus        231        671      0.246      0.499       0.29      0.192
          Pickup Truck         67        128          1          0     0.0323     0.0225
         Utility Truck        156        394     0.0903     0.0558     0.0299     0.0192
                 Truck        350       1611      0.263      0.225      0.143     0.0919
           Cargo Truc

array([    0.10632,     0.47964,     0.54341,    0.058578,   0.0035444,     0.44893,     0.19234,     0.02248,     0.01916,     0.09193,    0.059464,     0.16804,  0.00045004,    0.074875,    0.049635,    0.023394,   0.0012311,     0.13833,     0.36402,     0.30789,    0.018668,     0.25956,   0.0051265,    0.099201,
          0.047898,     0.07924,     0.10121,     0.22184,     0.13887,    0.061391,     0.39219,      0.1535,     0.30576,    0.013838,    0.005041,      0.1061,   0.0052702,    0.091178,    0.070872,      0.1029,     0.42328,    0.030176,     0.13364,     0.18541,    0.015814,    0.019348,   0.0015868,    0.031902,
           0.41787,    0.015681,    0.065943,     0.14197,    0.012918,    0.079379,     0.50111,     0.32078,     0.21059,    0.080343,     0.17411,   0.0045301])

In [13]:
# Validate the model
from ultralytics import YOLO

model = YOLO("../yolox/xView_2xGPU27/weights/epoch5.pt") # n for nano, x for huge
metrics = model.val(data="../datasets/xView/xview_yolo.yaml")  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category



Ultralytics 8.3.127 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA RTX 6000 Ada Generation, 48625MiB)
YOLO11x-obb summary (fused): 199 layers, 58,804,903 parameters, 0 gradients, 203.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1412.7±1366.8 MB/s, size: 111.4 KB)


val: Scanning /home/jupyter-dai7591/yolo/datasets/xView/labels/val.cache... 1300 images, 1117 backgrounds, 0 corrupt: 100%|██████████| 2417/2417 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 152/152 [00:36<00:00,  4.14it/s]


                   all       2417      69508       0.52      0.221      0.162     0.0971
   Fixed-wing Aircraft         12         12      0.242      0.333      0.185     0.0933
        Small Aircraft         13         31      0.654      0.452      0.514      0.349
 Passenger/Cargo Plane         58        110      0.655      0.736      0.759      0.545
            Helicopter          2          4      0.109        0.5     0.0564      0.041
     Passenger Vehicle         64        350          1          0    0.00694    0.00452
             Small Car        827      23836       0.59      0.758      0.704      0.407
                   Bus        231        671      0.212      0.402      0.229      0.139
          Pickup Truck         67        128          1          0     0.0214     0.0145
         Utility Truck        156        394     0.0794     0.0635     0.0306     0.0184
                 Truck        350       1611      0.166      0.377      0.132     0.0819
           Cargo Truc

array([   0.093339,     0.34878,     0.54526,    0.040983,   0.0045198,     0.40704,     0.13901,    0.014515,    0.018399,    0.081881,    0.043724,     0.13137,  0.00052145,    0.073055,    0.040083,           0,           0,    0.097082,     0.22619,     0.12073,           0,           0,           0,    0.098493,
          0.062864,    0.069011,    0.030206,    0.022084,    0.054109,    0.052056,     0.35409,    0.059602,     0.16299,    0.031977,   0.0039984,    0.057758,           0,    0.019521,    0.015195,     0.11448,     0.42721,   0.0066939,     0.10848,      0.1224,   0.0060356,    0.024643,   0.0030756,    0.037999,
           0.39849,    0.008916,    0.073277,    0.067603,    0.021175,    0.086876,     0.26858,     0.26142,    0.074477,    0.043753,     0.13953,   0.0093299])

In [14]:
# Validate the model
from ultralytics import YOLO

model = YOLO("../yolox/xView_2xGPU27/weights/epoch10.pt") # n for nano, x for huge
metrics = model.val(data="../datasets/xView/xview_yolo.yaml")  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category



Ultralytics 8.3.127 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA RTX 6000 Ada Generation, 48625MiB)
YOLO11x-obb summary (fused): 199 layers, 58,804,903 parameters, 0 gradients, 203.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 542.4±102.2 MB/s, size: 103.3 KB)


val: Scanning /home/jupyter-dai7591/yolo/datasets/xView/labels/val.cache... 1300 images, 1117 backgrounds, 0 corrupt: 100%|██████████| 2417/2417 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 152/152 [00:43<00:00,  3.48it/s]


                   all       2417      69508      0.468       0.22      0.184      0.112
   Fixed-wing Aircraft         12         12          1          0     0.0913      0.072
        Small Aircraft         13         31      0.703      0.382      0.511      0.325
 Passenger/Cargo Plane         58        110      0.583      0.736      0.695      0.512
            Helicopter          2          4          1          0     0.0394     0.0355
     Passenger Vehicle         64        350          1          0    0.00607    0.00423
             Small Car        827      23836      0.607      0.756      0.707      0.425
                   Bus        231        671      0.199       0.51      0.291       0.19
          Pickup Truck         67        128          1          0     0.0297     0.0215
         Utility Truck        156        394      0.136    0.00508     0.0202     0.0131
                 Truck        350       1611      0.198       0.26      0.116     0.0772
           Cargo Truc

array([   0.072045,     0.32453,     0.51171,    0.035477,   0.0042258,      0.4255,     0.18975,    0.021497,    0.013061,    0.077235,    0.055111,     0.15088,  0.00067539,    0.061387,    0.030503,           0,           0,     0.11203,     0.27075,     0.28279,           0,     0.22478,   0.0047601,    0.064846,
          0.075994,     0.10621,     0.07088,    0.071558,    0.051186,    0.012506,     0.34993,     0.12513,       0.105,   0.0072737,   0.0057424,    0.086763,           0,    0.028013,    0.049886,    0.084942,     0.49024,    0.020764,     0.07947,     0.17395,    0.007439,     0.01646,   0.0032541,    0.010872,
           0.40932,   0.0052959,    0.054695,     0.12057,   0.0077889,    0.084172,     0.39611,     0.26398,     0.14332,     0.09319,     0.12506,    0.051015])

In [15]:
# Validate the model
from ultralytics import YOLO

model = YOLO("../yolox/xView_2xGPU27/weights/epoch10.pt") # n for nano, x for huge
metrics = model.val(data="../datasets/xView/xview_yolo.yaml")  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category



Ultralytics 8.3.127 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA RTX 6000 Ada Generation, 48625MiB)
YOLO11x-obb summary (fused): 199 layers, 58,804,903 parameters, 0 gradients, 203.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 541.5±225.8 MB/s, size: 128.1 KB)


val: Scanning /home/jupyter-dai7591/yolo/datasets/xView/labels/val.cache... 1300 images, 1117 backgrounds, 0 corrupt: 100%|██████████| 2417/2417 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 152/152 [00:43<00:00,  3.46it/s]


                   all       2417      69508      0.468       0.22      0.184      0.112
   Fixed-wing Aircraft         12         12          1          0     0.0913      0.072
        Small Aircraft         13         31      0.703      0.382      0.511      0.325
 Passenger/Cargo Plane         58        110      0.583      0.736      0.695      0.512
            Helicopter          2          4          1          0     0.0394     0.0355
     Passenger Vehicle         64        350          1          0    0.00607    0.00423
             Small Car        827      23836      0.607      0.756      0.707      0.425
                   Bus        231        671      0.199       0.51      0.291       0.19
          Pickup Truck         67        128          1          0     0.0297     0.0215
         Utility Truck        156        394      0.136    0.00508     0.0202     0.0131
                 Truck        350       1611      0.198       0.26      0.116     0.0772
           Cargo Truc

array([   0.072045,     0.32453,     0.51171,    0.035477,   0.0042258,      0.4255,     0.18975,    0.021497,    0.013061,    0.077235,    0.055111,     0.15088,  0.00067539,    0.061387,    0.030503,           0,           0,     0.11203,     0.27075,     0.28279,           0,     0.22478,   0.0047601,    0.064846,
          0.075994,     0.10621,     0.07088,    0.071558,    0.051186,    0.012506,     0.34993,     0.12513,       0.105,   0.0072737,   0.0057424,    0.086763,           0,    0.028013,    0.049886,    0.084942,     0.49024,    0.020764,     0.07947,     0.17395,    0.007439,     0.01646,   0.0032541,    0.010872,
           0.40932,   0.0052959,    0.054695,     0.12057,   0.0077889,    0.084172,     0.39611,     0.26398,     0.14332,     0.09319,     0.12506,    0.051015])

In [16]:
# Validate the model
from ultralytics import YOLO

model = YOLO("../yolox/xView_2xGPU27/weights/last.pt") # n for nano, x for huge
metrics = model.val(data="../datasets/xView/xview_yolo.yaml")  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category



Ultralytics 8.3.127 🚀 Python-3.10.17 torch-2.5.1 CUDA:0 (NVIDIA RTX 6000 Ada Generation, 48625MiB)
YOLO11x-obb summary (fused): 199 layers, 58,804,903 parameters, 0 gradients, 203.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 997.2±1295.3 MB/s, size: 93.6 KB)


val: Scanning /home/jupyter-dai7591/yolo/datasets/xView/labels/val.cache... 1300 images, 1117 backgrounds, 0 corrupt: 100%|██████████| 2417/2417 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 152/152 [00:34<00:00,  4.34it/s]


                   all       2417      69508      0.435       0.29      0.263      0.178
   Fixed-wing Aircraft         12         12      0.741      0.241      0.267      0.194
        Small Aircraft         13         31      0.752      0.489      0.602      0.386
 Passenger/Cargo Plane         58        110      0.781      0.736      0.744      0.557
            Helicopter          2          4          0          0     0.0586     0.0521
     Passenger Vehicle         64        350          1          0    0.00467      0.003
             Small Car        827      23836      0.718      0.754      0.753      0.482
                   Bus        231        671      0.361      0.461       0.35      0.251
          Pickup Truck         67        128          1          0     0.0846     0.0643
         Utility Truck        156        394      0.385     0.0102     0.0425     0.0274
                 Truck        350       1611      0.275      0.262      0.169       0.12
           Cargo Truc

array([    0.19417,     0.38634,      0.5569,    0.052072,   0.0030031,      0.4823,     0.25122,    0.064283,     0.02744,     0.12047,    0.077904,     0.23956,   0.0010578,     0.12755,    0.058819,    0.025719,   0.0033246,     0.17758,     0.43917,     0.20629,    0.021817,     0.70231,    0.037516,     0.15313,
          0.073692,     0.14172,     0.12236,      0.3182,     0.23206,    0.040765,     0.38239,     0.20064,     0.38098,    0.025209,   0.0098551,      0.1568,    0.015877,    0.096707,    0.098597,     0.16403,     0.68365,     0.02355,     0.17995,     0.28128,    0.037636,    0.045195,    0.004553,    0.050213,
           0.46309,    0.016701,     0.10949,      0.1628,    0.016548,     0.10891,     0.58623,     0.34294,     0.22575,    0.093015,     0.30762,    0.046064])